In [ ]:
# default_exp tasks

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [17]:
# export
import os
from biocores.softwares.default import *
from biocores.softwares.bwa import Bwa
from biocores.softwares.bowtie import Bowtie
from biocores.softwares.bowtie2 import Bowtie2
from biocores.softwares.hisat2 import Hisat2
from biocores.softwares.samtools import Samtools
from create_reference.commands import *
from create_reference.utils import file_exists

In [12]:
# export

def task_get_file(link,filename):
    if file_exists(filename):
        return 'echo {filename} exists'.format(filename=filename)
    else:
        cmds = [
            cmd_mkdir(os.path.dirname(filename)),
            cmd_wget(link,filename)
        ]
        return '\n'.join(cmds)

def task_bwa_build_index(software,reference):
    bwa = Bwa(software,bwaDefault)
    cmds = [
            bwa.cmd_build_index(reference)
        ]
        
    return '\n'.join(cmds)

def task_bowtie_build_index(software,reference, prefix):
    bowtie=Bowtie(software,bowtieDefault)
    cmds = [
            cmd_mkdir(os.path.dirname(prefix)),
            bowtie.cmd_build_index(reference, prefix)
        ]
        
    return '\n'.join(cmds) 

def task_bowtie2_build_index(software,reference, prefix):
    bowtie2=Bowtie2(software,bowtie2Default)
    cmds = [
            cmd_mkdir(os.path.dirname(prefix)),
            bowtie2.cmd_build_index(reference, prefix)
        ]
        
    return '\n'.join(cmds) 

def task_samtools_build_index(software,reference):
    samtools = Samtools(software,samtoolsDefault)
    cmds = [
        samtools.cmd_faidx(reference)
    ]
    return '\n'.join(cmds)

def task_hisat2_build_index(software,reference,prefix,snp='',gtf=''):
    hisat2=Hisat2(software,hisat2Default)
    genome_ss=None
    genome_exon=None
    genome_genotype=None
    genome_snp=None
    
    cmds = [
        cmd_mkdir(os.path.dirname(prefix))
    ]
    if snp != '':
        cmds.append(hisat2.cmd_prepare_snp_ucsc(reference,snp,prefix))
        genome_snp=prefix+'.snp'
        genome_genotype=prefix+'.haplotype'
    if gtf != '':
        cmds.append(hisat2.cmd_prepare_exon_ss(gtf,prefix))
        genome_ss=prefix+'.ss'
        genome_exon=prefix+'.exon'

    cmds.append(hisat2.cmd_build_index(reference, prefix, genome_ss=genome_ss, genome_exon=genome_exon,
                        genome_genotype=genome_genotype, genome_snp=genome_snp))
    return '\n'.join(cmds)

def task_gunzip(gzipfile):
    return cmd_gunzip(gzipfile)

In [13]:
bwa='bwa'
bowtie='bowtie'
bowtie2='bowtie2'
hisat2='hisat2'
reference = 'genome.fasta'
bowtie_prefix = '/tmp/bowtie_index'
bowtie2_prefix='/tmp/bowtie2_index'
print(task_bwa_build_index(bwa,reference))
print(task_bowtie_build_index(bowtie,reference,bowtie_prefix))
print(task_bowtie2_build_index(bowtie2,reference,bowtie2_prefix))


bwa  index  genome.fasta
mkdir -p /tmp
bowtie-build    genome.fasta /tmp/bowtie_index
mkdir -p /tmp
bowtie2-build    genome.fasta /tmp/bowtie2_index


In [16]:
print(task_hisat2_build_index(hisat2,reference,'/tmp/hisat2_prefix','xxxx.snp','xxx.gtf'))
print(task_hisat2_build_index(hisat2,reference,'/tmp/hisat2_prefix','','xxx.gtf'))
print(task_hisat2_build_index(hisat2,reference,'/tmp/hisat2_prefix','xxxx.snp',''))

mkdir -p /tmp
awk 'BEGIN{OFS="\t"} {if($2 ~ /^chr/) {$2 = substr($2, 4)}; if($2 == "M") {$2 = "MT"} print}' xxxx.snp \
    > /tmp/hisat2_prefix_snp.tmp
hisat2_extract_snps_haplotypes_UCSC.py genome.fasta xxxx.snp /tmp/hisat2_prefix
hisat2_extract_splice_sites.py xxx.gtf > /tmp/hisat2_prefix.ss
hisat2_extract_exons.py xxx.gtf > /tmp/hisat2_prefix.exon
hisat2-build -p  16  genome.fasta  --ss /tmp/hisat2_prefix.ss --exon /tmp/hisat2_prefix.exon --haplotype /tmp/hisat2_prefix.haplotype --genome_snp /tmp/hisat2_prefix.snp /tmp/hisat2_prefix
mkdir -p /tmp
hisat2_extract_splice_sites.py xxx.gtf > /tmp/hisat2_prefix.ss
hisat2_extract_exons.py xxx.gtf > /tmp/hisat2_prefix.exon
hisat2-build -p  16  genome.fasta  --ss /tmp/hisat2_prefix.ss --exon /tmp/hisat2_prefix.exon /tmp/hisat2_prefix
mkdir -p /tmp
awk 'BEGIN{OFS="\t"} {if($2 ~ /^chr/) {$2 = substr($2, 4)}; if($2 == "M") {$2 = "MT"} print}' xxxx.snp \
    > /tmp/hisat2_prefix_snp.tmp
hisat2_extract_snps_haplotypes_UCSC.py genome.fasta xxxx.sn

In [ ]:
hs_link='ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
ss_link='ftp://ftp.ensembl.org/pub/release-99/fasta/sus_scrofa/dna/Sus_scrofa.Sscrofa11.1.dna.toplevel.fa.gz'
outdir='/tmp/'
species='homo_sapiens'
version='99'
filename='genome.fasta.gz'


In [ ]:
print(task_get_file(hs_link,filename))

echo No need to make dirs
wget -c ftp://ftp.ensembl.org/pub/release-99/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz -O genome.fasta.gz


In [ ]:
print(task_get_file(ss_link,filename))

echo No need to make dirs
wget -c ftp://ftp.ensembl.org/pub/release-99/fasta/sus_scrofa/dna/Sus_scrofa.Sscrofa11.1.dna.toplevel.fa.gz -O genome.fasta.gz


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_defaults.ipynb.
Converted 02_tasks.ipynb.
Converted 03_commands.ipynb.
Converted 04_pipelines.ipynb.
Converted 05_recipes.ipynb.
Converted index.ipynb.
